The following code maps the data set types to strings

In [1]:
val stringLabels = mapOf(
    0 to "T-shirt/top",
    1 to "Trousers",
    2 to "Pullover",
    3 to "Dress",
    4 to "Coat",
    5 to "Sandals",
    6 to "Shirt",
    7 to "Sneakers",
    8 to "Bag",
    9 to "Ankle boots",
)

The following code adds the KotlinDL API for the project

In [2]:
@file:DependsOn("org.jetbrains.kotlinx:kotlin-deeplearning-tensorflow:0.5.2")
%use kotlin-dl

The following code creates the model to be used for the data set

In [9]:
val model = Sequential.of(
    Input(28, 28, 1), // Input dimensions as 28x28 image of thickness 1
    org.jetbrains.kotlinx.dl.api.core.layer.reshaping.Flatten(), // Remap input to 1D list of bytes
    Dense(250), // Arbitrary choice of processing layer
    Dense(250), // Arbitrary choice of processing layer
    Dense(10), // Output layer
)

The following code creates the training and testing data

In [5]:
val (train, test) = fashionMnist() // Do it here to reference from everywhere else

Extracting 60000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-images-idx3-ubyte.gz
Extracting 60000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-labels-idx1-ubyte.gz
Extracting 10000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-images-idx3-ubyte.gz
Extracting 10000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-labels-idx1-ubyte.gz


The following code trains and tests the model

In [ ]:
import java.io.File

model.use {
    it.compile(
        optimizer = Adam(),
        loss = Losses.SOFT_MAX_CROSS_ENTROPY_WITH_LOGITS,
        metric = Metrics.ACCURACY
    ) // Compiles the model with the ADAM optimiser algorithm and uses accuracy as metric

    it.printSummary() // Prints out model information

    it.fit(
        dataset = train, // Use the train set to fit to the model
        epochs = 50, // Number of full cycles of data fitting
        batchSize = 200 // How big one batch of data should be
    )

    val accuracy = it.evaluate(dataset = test, batchSize = 200).metrics[Metrics.ACCURACY] // Evaluate model

    println("Accuracy: $accuracy") // Print out the accuracy
    it.save(File("model/my_model"), writingMode = WritingMode.OVERRIDE) // Save the model to a file to reference it later
}

Model type: Sequential
______________________________________________________________________________
Layer (type)                           Output Shape              Param #      
input_1(Input)                         [None, 28, 28, 1]         0            
______________________________________________________________________________
flatten_2(Flatten)                     [None, 784]               0            
______________________________________________________________________________
dense_3(Dense)                         [None, 250]               196250       
______________________________________________________________________________
dense_4(Dense)                         [None, 250]               62750        
______________________________________________________________________________
dense_5(Dense)                         [None, 10]                2510         
______________________________________________________________________________
Total trainable params: 26151

The following code is an example of how to load the model

In [4]:
import java.io.File

// val (train, test) = fashionMnist() done above, so don't repeat it here as expensive calculation

org.jetbrains.kotlinx.dl.api.inference.TensorFlowInferenceModel.load(File("model/my_model")).use {
    it.reshape(28, 28, 1)
    val prediction = it.predict(test.getX(5))
    val actualLabel = test.getY(5)

    println("Predicted label is: $prediction. This corresponds to class ${stringLabels[prediction]}.")
    println("Actual label is: $actualLabel.")
}

Extracting 60000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-images-idx3-ubyte.gz
Extracting 60000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-labels-idx1-ubyte.gz
Extracting 10000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-images-idx3-ubyte.gz
Extracting 10000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-labels-idx1-ubyte.gz
Predicted label is: 1. This corresponds to class Trousers.
Actual label is: 1.0.


The following code will go through the test data and plot the predicted results from the model

In [5]:
%use lets-plot

In [11]:
val `data` = mutableListOf<String>()

// val (train, test) = fashionMnist() done above, so don't repeat it here, as it is an expensive calculation

org.jetbrains.kotlinx.dl.api.inference.TensorFlowInferenceModel.load(File("model/my_model")).use {
    it.reshape(28, 28, 1)
    for (i in 0 until train.xSize()) {
        `data`.add(stringLabels[it.predict(train.getX(i)).toInt()]!!)
    }
    for (i in 0 until test.xSize()) {
        `data`.add(stringLabels[it.predict(test.getX(i)).toInt()]!!)
    }
}

val plot = letsPlot(mapOf("x" to `data`)) +
    geomBar(fill = "dark_green", color = "dark_green", alpha = 0.3) {
        x = "x"
    } + 
    scaleXDiscrete() + 
    scaleYContinuous()
    
(plot)

Extracting 60000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-images-idx3-ubyte.gz
Extracting 60000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-labels-idx1-ubyte.gz
Extracting 10000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-images-idx3-ubyte.gz
Extracting 10000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-labels-idx1-ubyte.gz


In [10]:
val `data` = mutableListOf<String>()

// val (train, test) = fashionMnist() done above, so don't do it here as expensive calculation

org.jetbrains.kotlinx.dl.api.inference.TensorFlowInferenceModel.load(File("model/my_model")).use {
    it.reshape(28, 28, 1)
    for (i in 0 until train.xSize()) {
        `data`.add(stringLabels[train.getY(i).toInt()]!!)
    }
    for (i in 0 until test.xSize()) {
        `data`.add(stringLabels[test.getY(i).toInt()]!!)
    }
}

val plot = letsPlot(mapOf("x" to `data`)) +
    geomBar(fill = "dark_green", color = "dark_green", alpha = 0.3) {
        x = "x"
        
    } + 
    scaleXDiscrete() + 
    scaleYContinuous()
    
(plot)

Extracting 60000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-images-idx3-ubyte.gz
Extracting 60000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\train-labels-idx1-ubyte.gz
Extracting 10000 images of 28x28 from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-images-idx3-ubyte.gz
Extracting 10000 labels from H:\Kotlin\FirstNeuralNetwork\cache\datasets\fashionmnist\t10k-labels-idx1-ubyte.gz
